#### The goal of this notebook is to query data from the Bitcoin blockchain in order extract a few on-chain insights. To do so, you'd be using [Google's BigQuery Bitcoin Database](https://cloud.google.com/blog/topics/public-datasets/bitcoin-in-bigquery-blockchain-analytics-on-public-data).

---

There are two ways in which you can interact with __Bitcoin's BigQuery DB__.

1. __If you have a Google Cloud Platform account, or you set one up, you can interact with the database directly.__

In order to be able to query data from `BigQuery`, you need to authenticate. Go to your GCP project and create a new project. Select `APIs & Services > Dashboard > Enable APIs and Services`. Click on Manage. Once enabled head to `APIs & Services > Credentials > Create credentials > Service account key > New service account`. Create a service account with a role (__owner__) that let's you query BigQuery data (e.g. BigQuery Data Admin) and download the key as JSON. Then simply define an environment variable that points to the JSON file, e.g.

There is more than one simpler way of making it working by explicity mentioning Credentials and passing them to client as shown below:

a)
> `export GOOGLE_APPLICATION_CREDENTIALS="/your/path/to/gcp_credentials.json"`

b)
> `os.environ['GOOGLE_APPLICATION_CREDENTIALS']="/your/path/to/gcp_credentials.json"`

Done! You should be able to make queries on the Bitcoin BigQuery database.

<br>
<br>

2. __You can also use Kaggle's public dataset BigQuery integration__.

There's a nice tutorial about `SQL` and `BigQueries` in this [notebook](https://www.kaggle.com/rtatman/sql-scavenger-hunt-handbook/) by Kaggle's Grandmaster Rachael Tatman.

___

In [2]:
# the BigQuery client library for Python provides a magic command that lets you run queries with minimal code
%load_ext google.cloud.bigquery

# importing libraries
import pandas as pd
#import pandas_gbq
from google.cloud import bigquery
#import bq_helper # unfortunetly this library is only available for kaggle's kernels
import os

### <font color= red> WARNING </font>: Be aware of malicious actors on the internet, DO NOT SHARE any personal information or keys on a public website like Github!

In [ ]:
# connecting to Google datastore:

os.environ['GOOGLE_APPLICATION_CREDENTIALS']="llave1.json"

In [20]:
# initialize a client

client = bigquery.Client()

#### BIG DISCLAIMER when running bigqueries

_Don't rely on __LIMIT__: One of the things that can be confusing when working with BigQuery datasets is the difference between the data you scan and the data you actually get back especially since it's the first one that actually counts against your quota. When you do something like select a column with LIMIT = 10, you'll only get 10 results back... but you'll actually be scanning the whole column. It's not a big deal if your table has 1000 rows, but it's a much bigger deal if it has 10,000,000 rows!_

__Since the monthly quota for BigQuery queries is 5 terabytes, you can easily go past your 30-day quota by running just a couple of queries!__

To put this into some perspective the table [`transactions`] of the crypto_bitcoin dataset on GCP is almost 2 TB... So be very careful when running your queries.

#### Explore a bit the dataset's summary [here](https://www.kaggle.com/bigquery/bitcoin-blockchain?select=transactions)


In [21]:
# query
# this size of this query is 1.42 TB

# Apr 12th - Apr 15th
# 678973 - 679422

# 679250 AND 679422 -> one day Apr 14th


query = """
    SELECT
        size, 
        block_number, 
        block_timestamp, 
        input_count, 
        output_count, 
        is_coinbase, 
        fee, 
        inputs, 
        outputs
        
    FROM `bigquery-public-data.crypto_bitcoin.transactions`
    WHERE block_number BETWEEN 679250 AND 679422
    ORDER BY block_number
    """

In [ ]:
# client.query() method runs the query

df = client.query(query).to_dataframe()

In [ ]:
df.to_csv('blocks.csv')